mainly looking at https://github.com/ngessert/isic2019/blob/master/models.py

# start

In [1]:
class Param: pass
param = Param()

param.PRFX = 'Mel0627A2'
param.PRFX_PREP = 'MelPrp0626A2'
param.ARCH = 'efficientnet-b1'
param.SZ = 224
param.EPOCHS = 10
param.BS = 128
param.K=5; param.SEED=101; param.FLD2USE=0
param.FP16 = True

param.LR=1e-3
param.WD=0.

param.N_SAMPL = int(5e3)

param.DEBUG = False
if param.DEBUG: 
    param.EPOCHS = 2
    param.K = 2
    param.N_SAMPL = 256

DEVICE = 'cuda'; PIN_MEM = (DEVICE=='cuda'); N_WORKERS=4



In [2]:
param.__dict__

{'PRFX': 'Mel0627A2',
 'PRFX_PREP': 'MelPrp0626A2',
 'ARCH': 'efficientnet-b1',
 'SZ': 224,
 'EPOCHS': 10,
 'BS': 128,
 'K': 5,
 'SEED': 101,
 'FLD2USE': 0,
 'FP16': True,
 'LR': 0.001,
 'WD': 0.0,
 'N_SAMPL': 5000,
 'DEBUG': False}

In [3]:
!nvidia-smi

Sat Jun 27 19:00:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   57C    P8    28W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# setup

In [4]:
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import random
import os, sys, gc
import datetime
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score as roc_auc

# https://github.com/eriklindernoren/PyTorch-YOLOv3/issues/162#issuecomment-491115265
from PIL import ImageFile; ImageFile.LOAD_TRUNCATED_IMAGES = True

def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

import torch
device=torch.device(DEVICE)
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler

from efficientnet_pytorch import EfficientNet

from apex import amp

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
set_seed(param.SEED)



p_out=f'../output/{param.PRFX}'; Path(p_out).mkdir(exist_ok=True,parents=True)
# p_cmp = '../input/siim-isic-melanoma-classification'



# prep

In [5]:
# train,test,sample_submission = (pd.read_csv(o, nrows=100 if DEBUG else None) 
#                                 for o in [f'{p_cmp}/{o}.csv' 
#                                           for o in ('train', 'test', 'sample_submission')])
# print([o.shape for o in (train,test,sample_submission)])
# display(test.head(2))
# p_19 = '../input/andrewmvd--isic-2019'

p_prp = f'../output/{param.PRFX_PREP}'
dftrn = pd.read_csv(f'{p_prp}/train_all.csv') 
print(dftrn.shape)

set_seed(param.SEED); dftrn = dftrn.sample(frac=1.)

if param.N_SAMPL is not None: dftrn = dftrn.head(param.N_SAMPL)
    
display(dftrn.head(2))
display(dftrn.shape)
display(dftrn.source.value_counts())
display(dftrn.groupby('source').target.mean())

(58457, 4)


,source,im_pth,patient_id,target
24437,20,../input/siim-isic-melanoma-classification/jpe...,IP_4021847,0.0
57432,19,../input/andrewmvd--isic-2019/ISIC_2019_Traini...,BCN_0004730,1.0


(5000, 4)

20    2780
19    2220
Name: source, dtype: int64

source
19    0.191892
20    0.016906
Name: target, dtype: float64

## folds

In [6]:
idx_nopid=np.where(dftrn.patient_id.isna())[0]
print(len(idx_nopid))
dftrn['patient_id'].iloc[idx_nopid]=[f'Nan_{i}' for i in range(len(idx_nopid))]
assert dftrn.patient_id.isna().mean()==0

174


In [7]:
set_seed(param.SEED)
kf = GroupKFold(n_splits=param.K)
fld2trvl={fld:(tr,vl) for fld,(tr,vl) in enumerate(kf.split(dftrn, groups=dftrn.patient_id))}

for fld, (tr, vl) in fld2trvl.items():
    print(fld, len(tr), len(vl))
    dftr=dftrn.iloc[tr]
    dfvl=dftrn.iloc[vl]
    assert set(dftr.patient_id)&set(dfvl.patient_id)==set()
for fld, (tr, vl) in fld2trvl.items():
    dftr=dftrn.iloc[tr]
    display(dftr.target.value_counts()/len(tr))
for fld, (tr, vl) in fld2trvl.items():
    dftr=dftrn.iloc[tr]
    display(dftr.source.value_counts())


0 4000 1000
1 4000 1000
2 4000 1000
3 4000 1000
4 4000 1000


0.0    0.90475
1.0    0.09525
Name: target, dtype: float64

0.0    0.9015
1.0    0.0985
Name: target, dtype: float64

0.0    0.90875
1.0    0.09125
Name: target, dtype: float64

0.0    0.90825
1.0    0.09175
Name: target, dtype: float64

...

20    2221
19    1779
Name: source, dtype: int64

20    2196
19    1804
Name: source, dtype: int64

20    2240
19    1760
Name: source, dtype: int64

20    2231
19    1769
Name: source, dtype: int64

20    2232
19    1768
Name: source, dtype: int64

## load images

In [8]:
# ims_pil = []
# for im_pth in tqdm(dftrn.im_pth):
#     ims_pil.append(Image.open(im_pth))
### faster to preload images
# i = np.random.choice(range(len(dftrn)))
# %%timeit 
# im_pil = Image.open(dftrn.im_pth.values[i])
# %%timeit 
# im_pil = ims_pil[i]

# dataset

In [9]:
class MelDataset(Dataset):
    def __init__(self, df, istrn=True):
        self.df = df
        self.istrn = istrn
        self.composed = transforms.Compose([
            transforms.RandomResizedCrop(param.SZ),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ColorJitter(brightness=32. / 255.,saturation=0.5),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])    
    def __getitem__(self, i):
        x = Image.open(self.df.im_pth.values[i]) #self.imgs[i]
        x = self.composed(x)
        if self.istrn:
            y = self.df.target.values[i]
            return x, y
        else:
            return (x,)
    
    def __len__(self):
        return len(self.df)    

In [10]:
# ds_trn = MelDataset(dftrn)
# ds_trn[0][0].shape, ds_trn[0][1]

# plt.imshow(ds_trn[0][0].numpy().transpose(1,2,0));
# plt.show()

# dl_trn = DataLoader(ds_trn, batch_size=param.BS, shuffle=True, num_workers=N_WORKERS, pin_memory=PIN_MEM)
# for dat in dl_trn:
#     x,y=dat; break

# print(x.shape, y)

# del ds_trn; gc.collect()

# model

In [11]:
# mdl = EfficientNet.from_pretrained(ARCH, num_classes=1)
# mdl(x).shape

# train

In [12]:
tr,vl=fld2trvl[param.FLD2USE]
dftr=dftrn.iloc[tr]
dfvl=dftrn.iloc[vl]
ystr=dftr.target.values
ysvl=dfvl.target.values
print(len(dftr), len(dfvl))
dstr = MelDataset(dftr)
dsvl = MelDataset(dfvl)
print(len(dstr), len(dsvl))
dltr = DataLoader(dstr, batch_size=param.BS,   shuffle=True,  num_workers=N_WORKERS, pin_memory=PIN_MEM)
dlvl = DataLoader(dsvl, batch_size=param.BS*2, shuffle=False, num_workers=N_WORKERS, pin_memory=PIN_MEM)
print(len(dltr), len(dlvl))
lendl=len(dltr)

4000 1000
4000 1000
32 4


In [13]:
def evaluate(model, dl):
    with torch.no_grad():
        model.eval()
        prd = []
        y = []
        for step, batch in enumerate(dl):
            xb, yb=(o.to(device) for o in batch)
            prdb = model(xb)
            prd.append(prdb.cpu().detach().numpy())
            y.append(yb.cpu().detach().numpy())
        prd = np.concatenate(prd)    
        y = np.concatenate(y)    
        lss = F.binary_cross_entropy_with_logits(torch.tensor(prd),torch.tensor(y).unsqueeze(1)).item()
        scr = roc_auc(y, prd)
    return lss, scr, y, prd

In [14]:
mdl = EfficientNet.from_pretrained(param.ARCH, num_classes=1)
mdl = mdl.to(device)

# opt = optim.SGD(mdl.parameters(), lr=param.LR, momentum=param.MOMENTUM, weight_decay=param.WD)
opt = optim.Adam(mdl.parameters(), lr=param.LR, weight_decay=param.WD)
# schdl = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=PLTFACTOR, patience=PATIENCE, min_lr=MIN_LR, verbose=True)

mdl, opt = amp.initialize(mdl, opt, opt_level='O1', verbosity=0)
mdl.zero_grad()


Loaded pretrained weights for efficientnet-b1


## run train

In [15]:
stats = {
    'lss': {'tr':[],'vl':[]},
    'scr': {'tr':[],'vl':[]},
}
oof = {'y':[], 'prd':[]}
best_scr = float('-inf')
best_epc = -1
for epc in range(param.EPOCHS):
    prdtr_ep=[]
    ytr_ep=[]
    for step, dat in enumerate(dltr):
        mdl.train()
        xb,yb=(o.to(device) for o in dat)
        yb = yb.unsqueeze(1)
        prdb = mdl(xb)
        loss = F.binary_cross_entropy_with_logits(prdb, yb)
        if param.FP16:
            with amp.scale_loss(loss, opt) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()
        if param.FP16:
            torch.nn.utils.clip_grad_norm_(amp.master_params(opt), 1)
        else:
            torch.nn.utils.clip_grad_norm_(mdl.parameters(), 1)
        opt.step()
        opt.zero_grad()
        prdtr_ep.append(prdb.cpu().detach().numpy())
        ytr_ep.append(yb.cpu().detach().numpy())
        if step%1000==0: 
            print(dtnow(), f'ep {epc} step {step}/{lendl}')
    
    prdtr_ep = np.concatenate(prdtr_ep)
    ytr_ep = np.concatenate(ytr_ep)    
    lss_tr_ep = F.binary_cross_entropy_with_logits(torch.tensor(prdtr_ep),torch.tensor(ytr_ep)).item()
    scr_tr_ep = roc_auc(ytr_ep, prdtr_ep)
    stats['lss']['tr'].append(lss_tr_ep)
    stats['scr']['tr'].append(scr_tr_ep)
    
    lss_vl_ep, scr_vl_ep, yvl_ep, prdvl_ep = evaluate(mdl, dlvl)
    stats['lss']['vl'].append(lss_vl_ep)
    stats['scr']['vl'].append(scr_vl_ep)
    oof['y'].append(yvl_ep)
    oof['prd'].append(prdvl_ep)
    
    print(dtnow(), f'ep {epc}: lss_tr {lss_tr_ep:.3f}; lss_vl {lss_vl_ep:.3f}; scr_tr {scr_tr_ep:.3f}; scr_vl {scr_vl_ep:.3f}; ')
    
    if scr_vl_ep>best_scr:
        print(f'better scr {best_scr:.3f} -> {scr_vl_ep:.3f}')
        best_scr = scr_vl_ep
        best_epc = epc
        if not param.DEBUG: torch.save(mdl.state_dict(), f'{p_out}/model_{param.PRFX}_fld_{fld}_best.p')
    if not param.DEBUG: torch.save(mdl.state_dict(), f'{p_out}/model_{param.PRFX}_fld_{fld}_epc_{epc}.p')
        
#     schdl.step(lossval)  # Update learning rate schedule
        
    

2020-06-27 19:02:03 ep 0 step 0/32
2020-06-27 19:11:03 ep 0: lss_tr 0.300; lss_vl 0.430; scr_tr 0.773; scr_vl 0.770; 
better scr -inf -> 0.770
2020-06-27 19:12:05 ep 1 step 0/32
2020-06-27 19:21:56 ep 1: lss_tr 0.235; lss_vl 0.316; scr_tr 0.850; scr_vl 0.846; 
better scr 0.770 -> 0.846
2020-06-27 19:22:57 ep 2 step 0/32
2020-06-27 19:32:43 ep 2: lss_tr 0.216; lss_vl 0.315; scr_tr 0.874; scr_vl 0.848; 
better scr 0.846 -> 0.848
2020-06-27 19:33:49 ep 3 step 0/32


KeyboardInterrupt: 

## run done

In [ ]:
print(f'best_scr {best_scr:.4f}')

In [ ]:
pd.DataFrame(stats['lss'])

In [ ]:
pd.DataFrame(stats['scr'])

In [ ]:
for k,v in stats.items():
    for tv,vs in v.items():
        plt.title(f'{k} {tv}')
        plt.plot(vs, 'o--')
    plt.show()

In [ ]:
stats['best_scr'] = best_scr
stats['best_epc'] = best_epc

results = {}
results['fld2trvl'] = fld2trvl
results['oof'] = oof
results['stats'] = stats

pickle.dump(results, open(f'{p_out}/results.p', 'wb'))

In [ ]:
pickle.dump(param, open(f'{p_out}/param.p', 'wb'))

# fin

In [ ]:
print(f'best_scr {best_scr:.4f}')

In [ ]:
param.__dict__

In [ ]:
!nvidia-smi